In [8]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Polynomial as P

In [9]:
def color_segmentation(image):
    # rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    lower_yellow = np.array([18, 95, 95], dtype=np.uint8)
    upper_yellow = np.array([105, 255, 255], dtype=np.uint8)

    lower_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_white = np.array([255, 80, 255], dtype=np.uint8)

    mask_yellow = cv2.inRange(image, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(image, lower_white, upper_white)

    combined_mask = cv2.bitwise_or(mask_yellow, mask_white)

    # segmented_image = cv2.bitwise_and(image, image, mask=combined_mask)
    # combined_mask = cv2.bitwise_or(mask_yellow, mask_white)
    inverted_mask = cv2.bitwise_not(combined_mask)
    image=cv2.cvtColor(image,cv2.COLOR_HSV2BGR)
    masked_color_image = np.full_like(image, [255,255,255] , dtype=np.uint8)

    # Combine the masked color image with the original image using the inverted mask
    segmented_image = cv2.bitwise_or(cv2.bitwise_and(image, image, mask=inverted_mask), 
                                      cv2.bitwise_and(masked_color_image, masked_color_image, mask=combined_mask))

    # img = cv2.bitwise_and(image, image, mask=cv2.bitwise_or(mask_white, mask_yellow))
    return segmented_image

In [10]:
def warp(img):
    img_size = (img.shape[1], img.shape[0])

    pt1 = np.float32([[685, 450],
          [1090, 710],
          [220, 710],
          [595, 450]])

    pt2 = np.float32(
        [[900, 0],
          [900, 710],
          [250, 710],
          [250, 0]])

    M = cv2.getPerspectiveTransform(pt1, pt2)
    Minv = cv2.getPerspectiveTransform(pt2, pt1)

    binary_warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    return binary_warped, Minv

In [11]:
def warp(img):
    img_size = (img.shape[1], img.shape[0])

    pt1 = np.float32([[685, 450],
          [1090, 710],
          [220, 710],
          [595, 450]])

    pt2 = np.float32(
        [[900, 0],
          [900, 710],
          [250, 710],
          [250, 0]])

    M = cv2.getPerspectiveTransform(pt1, pt2)
    Minv = cv2.getPerspectiveTransform(pt2, pt1)

    binary_warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    return binary_warped, Minv

In [12]:
def unwarp(warped_img, Minv):
    img_size = (warped_img.shape[1], warped_img.shape[0])
    unwarped_img = cv2.warpPerspective(warped_img, Minv, img_size, flags=cv2.INTER_LINEAR)
    return unwarped_img

In [13]:
def polyfitlines(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold, binary = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
    roi_x = binary.shape[1] // 2 
    roi_y_top = 0 
    roi_y_bottom = binary.shape[0] // 2  
    roi_width = binary.shape[1] // 2 
    roi_height = binary.shape[0] // 2  

    first_black_pixel_x_first_half = None
    first_black_pixel_y_first_half = None
    last_black_pixel_x_first_half = None
    last_black_pixel_y_first_half = None
    first_black_pixel_x_second_half = None
    first_black_pixel_y_second_half = None
    last_black_pixel_x_second_half = None
    last_black_pixel_y_second_half = None
    for y in range(roi_y_top, roi_y_bottom):
        for x in range(roi_x, roi_x + roi_width):
            intensity = binary[y, x]
            if intensity == 0:
                first_black_pixel_x_first_half = x
                first_black_pixel_y_first_half = y
                break
        else:
            continue
        break 
    
    for y in range(roi_y_bottom - 1, roi_y_top - 1, -1):
        for x in range(roi_x + roi_width - 1, roi_x - 1, -1):
            intensity = binary[y, x]
            if intensity == 0:
                last_black_pixel_x_first_half = x
                last_black_pixel_y_first_half = y
                break
        else:
            continue 
        break  

    for y in range(roi_y_bottom, roi_y_bottom + roi_height):
        for x in range(roi_x, roi_x + roi_width):
            intensity = binary[y, x]
            if intensity == 0:
                first_black_pixel_x_second_half = x
                first_black_pixel_y_second_half = y
                break
        else:
            continue 
        break

    for y in range(roi_y_bottom + roi_height - 1, roi_y_bottom - 1, -1):
        for x in range(roi_x + roi_width - 1, roi_x - 1, -1):
            intensity = binary[y, x]
            if intensity == 0:
                last_black_pixel_x_second_half = x
                last_black_pixel_y_second_half = y
                break
        else:
            continue 
        break

    points_first_half = np.array([[first_black_pixel_x_first_half, first_black_pixel_y_first_half],
                              [last_black_pixel_x_first_half, last_black_pixel_y_first_half]])

    points_second_half = np.array([[first_black_pixel_x_second_half, first_black_pixel_y_second_half],
                               [last_black_pixel_x_second_half, last_black_pixel_y_second_half]])


    combined_points = np.vstack((points_first_half, points_second_half))

    coefficients = np.polyfit(combined_points[:, 0], combined_points[:, 1], 1)
    poly = np.poly1d(coefficients)

    x_values = np.linspace(combined_points[0, 0], combined_points[-1, 0], 1000)

    y_values = poly(x_values)

    for i in range(len(x_values) - 1):
        cv2.line(image, (int(x_values[i]), int(y_values[i])), 
             (int(x_values[i+1]), int(y_values[i+1])), (255, 255, 255), 15)
    return image


In [14]:
def linesfit(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold, binary = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)


    roi_x = binary.shape[1] // 2  
    roi_y_top = 0  
    roi_y_bottom = binary.shape[0] // 2  
    roi_width = binary.shape[1] // 2  
    roi_height = binary.shape[0] // 2  

    first_black_pixel_x = None
    first_black_pixel_y = None
    last_black_pixel_x = None
    last_black_pixel_y = None

    for y in range(roi_y_top, roi_y_top + roi_height):
        for x in range(roi_x, roi_x + roi_width):
            intensity = binary[y, x]
            if intensity == 0:
                first_black_pixel_x = x
                first_black_pixel_y = y
                break
        else:
            continue  
        break 

    for y in range(roi_y_bottom + roi_height - 1, roi_y_bottom - 1, -1):
        for x in range(roi_x + roi_width - 1, roi_x - 1, -1):
            intensity = binary[y, x]

        
            if intensity == 0:
                last_black_pixel_x = x
                last_black_pixel_y = y
                break
        else:
            continue 
        break 
    cv2.line(image, (first_black_pixel_x, first_black_pixel_y), (last_black_pixel_x, last_black_pixel_y), (255, 255, 255), 11)
    return image

In [15]:
video = cv2.VideoCapture("videos/project_video.mp4")
# video = cv2.VideoCapture("videos/harder_challenge_video.mp4")
# video = cv2.VideoCapture("videos/challenge_video.mp4")
while True:
    ret, frame = video.read()
    if not ret:
        break
    base_img=frame
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame=cv2.GaussianBlur(frame, (3, 3), 0)
    # frame=cv2.Canny(frame,30, 130)
    wrapped,wrap2= warp(frame)
    f = color_segmentation(wrapped)
    # print((f.dtype))
    # f=cv2.cvtColor(f,cv2.COLOR_HSV2BGR)
    # print(f)
    # cv2.imshow('bianry',binary)
    # f=polyfitlines(f)
    f=linesfit(f)
    im=unwarp(f,wrap2)
    # cv2.imshow("final",image)
    cv2.imshow('Video', frame)
    # cv2.imshow('kernel',kerenel_edge)
    cv2.imshow('wrapped',wrapped)
    cv2.imshow('color',f)
    cv2.imshow('unwarped',im)
    # cv2.imshow('wrapped2',wrap2)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()